In [1]:
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import folium
import branca
import branca.colormap as cm
from folium import plugins
from folium.plugins import HeatMap
import geopandas as gpd
import seaborn as sns
import random
import os

pd.set_option('display.max_columns', None)

a = os.getcwd()
os.chdir(a)
a

'/opt/app-root/src/승순/Full table'

# 1.행정구역별 인구 데이터 로드

In [2]:
filename = "대전시 행정구역별 인구 with geometry.json"
file = open(filename)
data1 = gpd.read_file(file)
data1.head()

,구,동,총인구수,청소년,생산가능 인구,고령인구,cars_cnt,gid,geometry
0,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마846996,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마847995,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848996,"MULTIPOLYGON (((127.33094 36.19342, 127.33093 ..."
3,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848997,"MULTIPOLYGON (((127.33093 36.19432, 127.33093 ..."
4,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848998,"MULTIPOLYGON (((127.33093 36.19522, 127.33093 ..."


In [3]:
data1.columns

Index(['구', '동', '총인구수', '청소년', '생산가능 인구', '고령인구', 'cars_cnt', 'gid',
       'geometry'],
      dtype='object')

## 필요컬럼만 정렬하여 data1으로 설정 -> 생략해야겠음 ( geometry 빼는 순간 geopandas -> pandas 됨 )

In [4]:
data1 = data1[['구', '동', '총인구수', '청소년', '생산가능 인구', '고령인구', 'cars_cnt','gid']]
data1.head()

,구,동,총인구수,청소년,생산가능 인구,고령인구,cars_cnt,gid
0,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마846996
1,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마847995
2,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848996
3,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848997
4,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848998


# 2. 지리적 특성 데이터 로드

In [5]:
filename = "0328-아침 격자.json"
file = open(filename)
data2 = gpd.read_file(file)
data2.head() #54912 rows

,gid,인구정보(총인구),인구정보(고령),인구정보(생산가능),인구정보(유소년),등록차량,보행 신호등 수,차량 신호등 수,안전지대 수,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3,정차금지지대 수,cctv 수,횡단보도 수,신형 횡단보도,구형 횡단보도,중앙분리대 수,교통링크 수,안전표지판 수,표지판-정주식,표지판-내민식,표지판-부착식,표지판-문형식,표지판종류-주의,표지판종류-규제,표지판종류-지시,표지판종류-보조,중앙분리대 면적,안전지대 면적,횡단보도 면적,정차금지지대 면적,geometry
0,다마846996,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,다마847996,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ..."
3,다마847997,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ..."
4,다마847998,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ..."


# 19번 데이터 기준 geometry & gid 데이터 (data3)

In [6]:
filename = "doro.json"
file = open(filename)
data3 = gpd.read_file(file)
data3.head() #12153 rows

,index,gid,acci_cnt,geometry
0,0,다바866110,0,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,1,다바823157,0,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,2,다바928260,0,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,3,다바931203,2,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,4,다바836109,0,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."


## data2와 data3을 병합하여 19번 데이터 기준 지리특성자료 생성 - data4

In [7]:
data4 = pd.merge(data2, data3)
data4.head() #12153 rows

,gid,인구정보(총인구),인구정보(고령),인구정보(생산가능),인구정보(유소년),등록차량,보행 신호등 수,차량 신호등 수,안전지대 수,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3,정차금지지대 수,cctv 수,횡단보도 수,신형 횡단보도,구형 횡단보도,중앙분리대 수,교통링크 수,안전표지판 수,표지판-정주식,표지판-내민식,표지판-부착식,표지판-문형식,표지판종류-주의,표지판종류-규제,표지판종류-지시,표지판종류-보조,중앙분리대 면적,안전지대 면적,횡단보도 면적,정차금지지대 면적,geometry,index,acci_cnt
0,다바778093,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.25280 36.28073, 127.25280 ...",21536,0
1,다바779092,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,1,0,1,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.25392 36.27983, 127.25392 ...",20648,0
2,다바779093,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.25392 36.28074, 127.25391 ...",21986,0
3,다바780091,0.0,0.0,0.0,0.0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.25504 36.27894, 127.25503 ...",47842,0
4,다바780092,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,6,3,0,3,0,1,3,1,1,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.25503 36.27984, 127.25503 ...",28027,0


In [8]:
data4.columns

Index(['gid', '인구정보(총인구)', '인구정보(고령)', '인구정보(생산가능)', '인구정보(유소년)', '등록차량',
       '보행 신호등 수', '차량 신호등 수', '안전지대 수', '속도표시 총 개수', '속도표시 종류 수', '속도표시-1',
       '속도표시-2', '속도표시-3', '정차금지지대 수', 'cctv 수', '횡단보도 수', '신형 횡단보도',
       '구형 횡단보도', '중앙분리대 수', '교통링크 수', '안전표지판 수', '표지판-정주식', '표지판-내민식',
       '표지판-부착식', '표지판-문형식', '표지판종류-주의', '표지판종류-규제', '표지판종류-지시', '표지판종류-보조',
       '중앙분리대 면적', '안전지대 면적', '횡단보도 면적', '정차금지지대 면적', 'geometry', 'index',
       'acci_cnt'],
      dtype='object')

# 필요한 컬럼만 추려서 data4로 재설정
구 & 동별 인구데이터는 data1에서 쓸것이므로 포함시키지 않음
추후 data1과 합칠예정이므로 중복되는 컬럼은 제거

In [9]:
data4 = data4[['등록차량','보행 신호등 수', '차량 신호등 수', '안전지대 수', '속도표시 총 개수', '속도표시 종류 수', '속도표시-1',
       '속도표시-2', '속도표시-3', '정차금지지대 수', 'cctv 수', '횡단보도 수', '신형 횡단보도',
       '구형 횡단보도', '중앙분리대 수', '교통링크 수', '안전표지판 수', '표지판-정주식', '표지판-내민식',
       '표지판-부착식', '표지판-문형식', '표지판종류-주의', '표지판종류-규제', '표지판종류-지시', '표지판종류-보조',
       '중앙분리대 면적', '안전지대 면적', '횡단보도 면적', '정차금지지대 면적','acci_cnt','gid', 'geometry']]

data4.head(3)

,등록차량,보행 신호등 수,차량 신호등 수,안전지대 수,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3,정차금지지대 수,cctv 수,횡단보도 수,신형 횡단보도,구형 횡단보도,중앙분리대 수,교통링크 수,안전표지판 수,표지판-정주식,표지판-내민식,표지판-부착식,표지판-문형식,표지판종류-주의,표지판종류-규제,표지판종류-지시,표지판종류-보조,중앙분리대 면적,안전지대 면적,횡단보도 면적,정차금지지대 면적,acci_cnt,gid,geometry
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바778093,"MULTIPOLYGON (((127.25280 36.28073, 127.25280 ..."
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,1,0,1,0,0,0,0.0,0.0,0.0,0.0,0,다바779092,"MULTIPOLYGON (((127.25392 36.27983, 127.25392 ..."
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바779093,"MULTIPOLYGON (((127.25392 36.28074, 127.25391 ..."


추후 데이터 추출시 to_csv하기 위해 데이터 형변환

In [ ]:
#data1['geometry'] = data1['geometry'].astype(str)
#data3['geometry'] = data3['geometry'].astype(str)

In [10]:
data1.head(3)

,구,동,총인구수,청소년,생산가능 인구,고령인구,cars_cnt,gid
0,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마846996
1,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마847995
2,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848996


In [11]:
data4.head() #12900 

,등록차량,보행 신호등 수,차량 신호등 수,안전지대 수,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3,정차금지지대 수,cctv 수,횡단보도 수,신형 횡단보도,구형 횡단보도,중앙분리대 수,교통링크 수,안전표지판 수,표지판-정주식,표지판-내민식,표지판-부착식,표지판-문형식,표지판종류-주의,표지판종류-규제,표지판종류-지시,표지판종류-보조,중앙분리대 면적,안전지대 면적,횡단보도 면적,정차금지지대 면적,acci_cnt,gid,geometry
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바778093,"MULTIPOLYGON (((127.25280 36.28073, 127.25280 ..."
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,1,0,1,0,0,0,0.0,0.0,0.0,0.0,0,다바779092,"MULTIPOLYGON (((127.25392 36.27983, 127.25392 ..."
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바779093,"MULTIPOLYGON (((127.25392 36.28074, 127.25391 ..."
3,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바780091,"MULTIPOLYGON (((127.25504 36.27894, 127.25503 ..."
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,6,3,0,3,0,1,3,1,1,0.0,0.0,0.0,0.0,0,다바780092,"MULTIPOLYGON (((127.25503 36.27984, 127.25503 ..."


In [12]:
final = pd.merge(data4, data1)
final.head() # 12119 rows

,등록차량,보행 신호등 수,차량 신호등 수,안전지대 수,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3,정차금지지대 수,cctv 수,횡단보도 수,신형 횡단보도,구형 횡단보도,중앙분리대 수,교통링크 수,안전표지판 수,표지판-정주식,표지판-내민식,표지판-부착식,표지판-문형식,표지판종류-주의,표지판종류-규제,표지판종류-지시,표지판종류-보조,중앙분리대 면적,안전지대 면적,횡단보도 면적,정차금지지대 면적,acci_cnt,gid,geometry,구,동,총인구수,청소년,생산가능 인구,고령인구,cars_cnt
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바778093,"MULTIPOLYGON (((127.25280 36.28073, 127.25280 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,1,0,1,0,0,0,0.0,0.0,0.0,0.0,0,다바779092,"MULTIPOLYGON (((127.25392 36.27983, 127.25392 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바779093,"MULTIPOLYGON (((127.25392 36.28074, 127.25391 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0
3,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바780091,"MULTIPOLYGON (((127.25504 36.27894, 127.25503 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,6,3,0,3,0,1,3,1,1,0.0,0.0,0.0,0.0,0,다바780092,"MULTIPOLYGON (((127.25503 36.27984, 127.25503 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0


## 이거 rename 생략할게!!

In [ ]:
final.rename(columns = { '구':'gu', '동':'dong', '총인구수':'pop_tot' , '청소년':'pop_teen','생산가능 인구':'pop_pro','고령인구':'pop_old','등록차량':'cars_cnt'
                                    ,'보행 신호등 수':'cross_traffice','차량 신호등 수':'car_traffice','안전지대 수':'safezone','속도표시 총 개수':'speed_tot',
                                    '속도표시 종류 수':'speed_num', '속도표시-1':'speed_knd1','속도표시-2':'speed_knd2','속도표시-3':'speed_knd3','정차금지지대 수':'stop_num',
                                    'cctv 수':'cctv_num', '횡단보도 수':'crosswalk_num','신형 횡단보도':'cross_new','구형 횡단보도':'cross_old','중앙분리대 수':'split_num',
                                    '교통링크 수':'link_num','안전표지판 수':'sign_num','표지판-정주식':'sign_stand','표지판-내민식':'sign_hang',
                                    '표지판-부착식':'sign_stick','표지판-문형식':'sign_door','표지판종류-주의':'sign_caution','표지판종류-규제':'sign_res',
                                    '표지판종류-지시':'sign_dir','표지판종류-보조':'sign_sub','중앙분리대 면적':'split_size','안전지대 면적':'safezone_size',
                                    '횡단보도 면적':'cross_size','정차금지지대 면적':'stop_size','acci_cnt':'acci_cnt','km':'km'},inplace=True)

In [ ]:
import folium
converted_json = final.to_json()
m = folium.Map(
    location=[36.34156,127.39704],
    zoom_start=11
)

folium.GeoJson(
    converted_json,
).add_to(m)

m

In [13]:
final.columns

Index(['등록차량', '보행 신호등 수', '차량 신호등 수', '안전지대 수', '속도표시 총 개수', '속도표시 종류 수',
       '속도표시-1', '속도표시-2', '속도표시-3', '정차금지지대 수', 'cctv 수', '횡단보도 수', '신형 횡단보도',
       '구형 횡단보도', '중앙분리대 수', '교통링크 수', '안전표지판 수', '표지판-정주식', '표지판-내민식',
       '표지판-부착식', '표지판-문형식', '표지판종류-주의', '표지판종류-규제', '표지판종류-지시', '표지판종류-보조',
       '중앙분리대 면적', '안전지대 면적', '횡단보도 면적', '정차금지지대 면적', 'acci_cnt', 'gid',
       'geometry', '구', '동', '총인구수', '청소년', '생산가능 인구', '고령인구', 'cars_cnt'],
      dtype='object')

In [ ]:
#train.to_csv('fulltable(ver1_0331).csv')

# y(target) 테이블 만들기

## 코멘트 추가(윤모) : doro.json 파일에 acci_cnt 합쳐져 있었어서 안해도 되는 과정이었음!
## 어쨌든 두 개 둘 다 해봐서 시각화해본 결과 : 결과가 똑같았음 (안해도 됐다!)
## 그래도 확인용으로 남겨둠

데이터 로드

In [14]:
filename = "2.대전광역시_교통사고격자(2017~2019).geojson"
file = open(filename)
df2 = gpd.read_file(file)
df2.head()

,gid,acci_cnt,geometry
0,다바866110,0,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,다바823157,0,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,다바928260,0,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,다바931203,2,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,다바836109,0,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."


In [15]:
df3 = df2[['acci_cnt','gid', 'geometry']]
df3.head()

,acci_cnt,gid,geometry
0,0,다바866110,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,0,다바823157,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,0,다바928260,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,2,다바931203,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,0,다바836109,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."


In [16]:
fulltable = pd.merge(final, df3)
fulltable.head()

,등록차량,보행 신호등 수,차량 신호등 수,안전지대 수,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3,정차금지지대 수,cctv 수,횡단보도 수,신형 횡단보도,구형 횡단보도,중앙분리대 수,교통링크 수,안전표지판 수,표지판-정주식,표지판-내민식,표지판-부착식,표지판-문형식,표지판종류-주의,표지판종류-규제,표지판종류-지시,표지판종류-보조,중앙분리대 면적,안전지대 면적,횡단보도 면적,정차금지지대 면적,acci_cnt,gid,geometry,구,동,총인구수,청소년,생산가능 인구,고령인구,cars_cnt
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바778093,"MULTIPOLYGON (((127.25280 36.28073, 127.25280 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,1,0,1,0,0,0,0.0,0.0,0.0,0.0,0,다바779092,"MULTIPOLYGON (((127.25392 36.27983, 127.25392 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바779093,"MULTIPOLYGON (((127.25392 36.28074, 127.25391 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0
3,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바780091,"MULTIPOLYGON (((127.25504 36.27894, 127.25503 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,6,3,0,3,0,1,3,1,1,0.0,0.0,0.0,0.0,0,다바780092,"MULTIPOLYGON (((127.25503 36.27984, 127.25503 ...",유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0


In [ ]:
import folium
converted_json = df3[df3['acci_cnt']>10].to_json() # 원본 격자
m = folium.Map(
    location=[36.34156,127.39704],
    zoom_start=11
)

folium.GeoJson(
    converted_json,
).add_to(m)

m

In [ ]:
import folium
converted_json = final[final['acci_cnt']>10].to_json() # final (df3이랑 합치기 전)
m = folium.Map(
    location=[36.34156,127.39704],
    zoom_start=11
)

folium.GeoJson(
    converted_json,
).add_to(m)

m

In [ ]:
import folium
converted_json = fulltable.to_json() # fulltable (df3이랑 합친 후)
m = folium.Map(
    location=[36.34156,127.39704],
    zoom_start=11
)

folium.GeoJson(
    converted_json,
).add_to(m)

m

In [17]:
fulltable.columns

Index(['등록차량', '보행 신호등 수', '차량 신호등 수', '안전지대 수', '속도표시 총 개수', '속도표시 종류 수',
       '속도표시-1', '속도표시-2', '속도표시-3', '정차금지지대 수', 'cctv 수', '횡단보도 수', '신형 횡단보도',
       '구형 횡단보도', '중앙분리대 수', '교통링크 수', '안전표지판 수', '표지판-정주식', '표지판-내민식',
       '표지판-부착식', '표지판-문형식', '표지판종류-주의', '표지판종류-규제', '표지판종류-지시', '표지판종류-보조',
       '중앙분리대 면적', '안전지대 면적', '횡단보도 면적', '정차금지지대 면적', 'acci_cnt', 'gid',
       'geometry', '구', '동', '총인구수', '청소년', '생산가능 인구', '고령인구', 'cars_cnt'],
      dtype='object')

In [19]:
fulltable.isna().head()

,등록차량,보행 신호등 수,차량 신호등 수,안전지대 수,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3,정차금지지대 수,cctv 수,횡단보도 수,신형 횡단보도,구형 횡단보도,중앙분리대 수,교통링크 수,안전표지판 수,표지판-정주식,표지판-내민식,표지판-부착식,표지판-문형식,표지판종류-주의,표지판종류-규제,표지판종류-지시,표지판종류-보조,중앙분리대 면적,안전지대 면적,횡단보도 면적,정차금지지대 면적,acci_cnt,gid,geometry,구,동,총인구수,청소년,생산가능 인구,고령인구,cars_cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [20]:
fulltable.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 12119 entries, 0 to 12118
Data columns (total 39 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   등록차량       12119 non-null  int64   
 1   보행 신호등 수   12119 non-null  int64   
 2   차량 신호등 수   12119 non-null  int64   
 3   안전지대 수     12119 non-null  int64   
 4   속도표시 총 개수  12119 non-null  int64   
 5   속도표시 종류 수  12119 non-null  int64   
 6   속도표시-1     12119 non-null  int64   
 7   속도표시-2     12119 non-null  int64   
 8   속도표시-3     12119 non-null  int64   
 9   정차금지지대 수   12119 non-null  int64   
 10  cctv 수     12119 non-null  int64   
 11  횡단보도 수     12119 non-null  int64   
 12  신형 횡단보도    12119 non-null  int64   
 13  구형 횡단보도    12119 non-null  int64   
 14  중앙분리대 수    12119 non-null  int64   
 15  교통링크 수     12119 non-null  int64   
 16  안전표지판 수    12119 non-null  int64   
 17  표지판-정주식    12119 non-null  int64   
 18  표지판-내민식    12119 non-null  int64   
 19  표지판-부착식    12119 

In [ ]:
fulltable.to_file("full_t(ver2_0427).json", driver="GeoJSON") # 저장

In [21]:
filename = "full_t(ver2_0427).json" # 확인
file = open(filename)
test = gpd.read_file(file)
test.head()

,등록차량,보행 신호등 수,차량 신호등 수,안전지대 수,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3,정차금지지대 수,cctv 수,횡단보도 수,신형 횡단보도,구형 횡단보도,중앙분리대 수,교통링크 수,안전표지판 수,표지판-정주식,표지판-내민식,표지판-부착식,표지판-문형식,표지판종류-주의,표지판종류-규제,표지판종류-지시,표지판종류-보조,중앙분리대 면적,안전지대 면적,횡단보도 면적,정차금지지대 면적,acci_cnt,gid,구,동,총인구수,청소년,생산가능 인구,고령인구,cars_cnt,geometry
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바778093,유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0,"MULTIPOLYGON (((127.25280 36.28073, 127.25280 ..."
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,1,0,1,0,0,0,0.0,0.0,0.0,0.0,0,다바779092,유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0,"MULTIPOLYGON (((127.25392 36.27983, 127.25392 ..."
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바779093,유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0,"MULTIPOLYGON (((127.25392 36.28074, 127.25391 ..."
3,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,다바780091,유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0,"MULTIPOLYGON (((127.25504 36.27894, 127.25503 ..."
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,6,3,0,3,0,1,3,1,1,0.0,0.0,0.0,0.0,0,다바780092,유성구,진잠동,36085.0,9968.0,26652.0,4148.0,0,"MULTIPOLYGON (((127.25503 36.27984, 127.25503 ..."
